Imports libs

In [4]:
from faker import Faker

from pathlib import Path 

import json
import random 
import datetime

import os
import sys

import requests

import pandas as pd 
import numpy as np

from tqdm import tqdm

import uuid


import plotly
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected = True)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

from joblib import Parallel, delayed

GLOBAL VARIABLES 

In [2]:
SIMULATTE_BASE_DIR = "/Users/tiemokodembele/Documents/internShip/simulatte-public" #os.environ['SIMULATTE_BASE_DIR'] 
SIMULATTE_ONLINE_HOSTNAME = "localhost" #os.environ['SIMULATTE_ONLINE_HOSTNAME'] 
SIMULATTE_ANALYTIC_HOSTNAME = "localhost" #os.environ['SIMULATTE_ANALYTIC_HOSTNAME'] 

Read __input data__ from __remote file__ by __url__

In [51]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen
import requests

import json
import os 

SIMULATTE_BASE_DIR = os.environ['SIMULATTE_BASE_DIR'] #"/Users/tiemokodembele/Documents/internShip/simulatte"

size = 10
filesize = size
if(size>999): filesize = str(size)[:-3]+"K"

payload = dict()
payload["data_sink_size"] = size
payload["data_source_url"] = "http://raw.githubusercontent.com/DecisionsDev/decisions-on-spark/master/data/miniloan/miniloan-decisions-100K.csv"
payload["data_sink_uri"] = f"{SIMULATTE_BASE_DIR}/data/ODM/loanvalidation/odm-loanvalidation-requests-{filesize}.json"

serverDjangoEndpoint = f"http://{SIMULATTE_ANALYTIC_HOSTNAME}:8000/data/from-url/" #"http://127.0.0.1:8000/data/from-url/"
headers = {'Content-type': 'application/json'} # 'Accept': 'text/plain'

############## DATA FROM REMOTE CSV FILE ################
try:    
    response = requests.post(serverDjangoEndpoint, data = json.dumps(payload), headers=headers)
    print("RESPONSE STATUS : ", response.status_code, "\n")
    if(response.status_code==200):
        print("RESPONSE CONTENT : ", response.content, "\n")
    else:
        print("Check Django server side. ERROR : ", response.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

RESPONSE STATUS :  200 

RESPONSE CONTENT :  b'Ready to get data from : http://raw.githubusercontent.com/DecisionsDev/decisions-on-spark/master/data/miniloan/miniloan-decisions-100K.csv et print in /Users/tiemokodembele/Documents/internShip/simulatte/data/ODM/loanvalidation/odm-loanvalidation-requests-10.json' 



Read __input data__ from __data_source_path__ in the local file system

In [52]:
import os
import sys

import requests

import pandas as pd 
import numpy as np
import json

import tqdm

######## GET DATA #########
dataSourcePathODM = payload["data_sink_uri"]

inputData = []
with open(dataSourcePathODM) as file:
    while (line := file.readline().rstrip()):
        inputData.append(json.loads(line))

data = pd.json_normalize(inputData, max_level=1)
print(data.shape[0])
data.head(5)


10


,borrower.name,borrower.creditScore,borrower.yearlyIncome,loan.amount,loan.duration,loan.yearlyInterestRate,loan.yearlyRepayment
0,Dr. Selena Smith,736,113243,783440,162,0.030005,70648
1,Casey Martin,527,224186,1787385,183,0.054819,173197
2,Ronald Medina,460,39954,1733494,75,0.055183,328563
3,Jordan Pace,751,219998,1350004,72,0.043400,255970
4,Jonathan Mccullough DDS,684,72470,1210944,160,0.037772,115742


Decision service Test

In [53]:
headers = {'Content-type': 'application/json'} # 'Accept': 'text/plain'

loanRequest = inputData[0]

odmEndpoint = f"http://{ODM_HOSTNAME}:9060/DecisionService/rest/miniloan/1.0/Miniloan_ServiceRuleset"

############## DECISION TEST ################
try:    
    odmResponse = requests.post(odmEndpoint, data = json.dumps(loanRequest), headers=headers)
    print("RESPONSE STATUS : ", odmResponse.status_code, "\n")
    if(odmResponse.status_code==200):
        print("RESPONSE CONTENT : ", json.dumps(odmResponse.json(), indent=5), "\n")
    else:
        print("Check ODM server side. ERROR : ", odmResponse.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

RESPONSE STATUS :  200 

RESPONSE CONTENT :  {
     "__DecisionID__": "6b561d65-c5c4-4fc3-947c-b61c131561a10",
     "loan": {
          "amount": 783440,
          "duration": 162,
          "yearlyInterestRate": 0.03000493351324669,
          "yearlyRepayment": 70648,
          "approved": false,
          "messages": [
               "Too big Debt-To-Income ratio",
               "debt-to-income too high compared to credit score"
          ]
     }
} 



CREATE SIMULATION 

In [54]:
datasinkFolderPath = f"{SIMULATTE_BASE_DIR}/data/ODM/loanvalidation"
createSimulationUrl = f"http://{SIMULATTE_ONLINE_HOSTNAME}:8080/v1/simulation/create"
simulationObject = {
    "name": "my simu",
    "description": "string",
    "createDate": "2022-05-18T23:31:19.828Z",
    "trace": True,
    "dataSource": {
      "format": "JSON",
      "uri": dataSourcePathODM,
      "username": "string",
      "password": "string"
    },
    "dataSink": {
      "format": "JSON",
      "folderPath": datasinkFolderPath,
      "username": "string",
      "password": "string"
    },
    "decisionService": {
      "type": "ODM",
      "endPoint": odmEndpoint,
      "authType": "NO_AUTH",
      "username": "string",
      "password": "string",
      "key": "string",
      "value": "string",
      "token": "string",
      "headerPrefix": "string"
    },
    "metrics": [
      {
        "uid": 0,
        "name": "string",
        "type": "SPARK_SQL",
        "description": "string",
        "expression": "string"
      }
    ],
    "kpi": [
      {
        "uid": 0,
        "name": "string",
        "type": "SPARK_SQL",
        "description": "string",
        "result": {}
      }
    ]
  }



############## CREATE SIMULATION ################
try:    
    createSimulation = requests.post(createSimulationUrl, data = json.dumps(simulationObject), headers=headers)
    print("RESPONSE STATUS : ", createSimulation.status_code, "\n")
    if(createSimulation.status_code==200):
        print("RESPONSE CONTENT : ", json.dumps(createSimulation.json(), indent=5), "\n")
    else:
        print("Check server side. ERROR : ", createSimulation.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)


RESPONSE STATUS :  200 

RESPONSE CONTENT :  {
     "uid": 22,
     "userUid": 0,
     "name": "my simu",
     "description": "string",
     "createDate": "2022-07-28T13:16:40.697+00:00",
     "lastUpdateDate": "2022-07-28T13:16:40.697+00:00",
     "trace": true,
     "dataSource": {
          "format": "JSON",
          "uri": "/Users/tiemokodembele/Documents/internShip/simulatte/data/ODM/loanvalidation/odm-loanvalidation-requests-10.json",
          "username": "string",
          "password": "string",
          "uid": 121
     },
     "dataSink": {
          "format": "JSON",
          "folderPath": "/Users/tiemokodembele/Documents/internShip/simulatte/data/ODM/loanvalidation",
          "uri": null,
          "username": "string",
          "password": "string",
          "uid": 120
     },
     "decisionService": {
          "type": "ODM",
          "endPoint": "http://localhost:9060/DecisionService/rest/miniloan/1.0/Miniloan_ServiceRuleset",
          "authType": "NO_AUTH",
     

START SIMULATION

In [76]:
simulationUid = createSimulation.json()["uid"]
startSimulationUrl = f"http://{SIMULATTE_ONLINE_HOSTNAME}:8080/v1/simulations/{simulationUid}/start/"
notebook_uri = f"{SIMULATTE_BASE_DIR}/notebooks/ODM/simulyticsOdm.ipynb"

run_config = {
    "name": "miniloan",
    "description": "string",
    "createDate": "2022-05-18T23:31:19.828Z",
    "trace": False,
    "dataSink": {
      "format": "JSON", 
      "folderPath": datasinkFolderPath,
      "username": "string",
      "password": "string"
    },
    "decisionService": {
      "type": "ODM",
      "endPoint": odmEndpoint,
      "authType": "NO_AUTH",
      "username": "string",
      "password": "string",
      "key": "string",
      "value": "string",
      "token": "string",
      "headerPrefix": "string"
    },
    "executor": {
        "type": "JSE",
        "mode": "REMOTE",
        "capability": "ODM",
        "uid": 0
    },
    "notebookUri": notebook_uri
  }


############## START SIMULATION ################
try:    
    startSimulation = requests.post(startSimulationUrl, data = json.dumps(run_config), headers=headers)
    print("RESPONSE STATUS : ", startSimulation.status_code, "\n")
    if(startSimulation.status_code==200):
        print("YOUR SIMULATION IS STARTED !!! \n")
        print("RESPONSE CONTENT : ", json.dumps(startSimulation.json(), indent=5), "\n")
    else:
        print("Check server side. ERROR : ", startSimulation.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

RESPONSE STATUS :  500 

Check server side. ERROR :  org.apache.spark.util.Utils$.classForName(Ljava/lang/String;)Ljava/lang/Class;


GET RUN REPORT

In [ ]:
runReportUrl = "http://simulation-server:8080/v1/simulations/{}/runs/{}/actions/get_report"
simulationUid = startSimulation.json()["simulationUid"]
runUid = startSimulation.json()["uid"]

############## RUN REPORT ################
try:    
    runReport = requests.post(runReportUrl.format(simulationUid, runUid))
    print("RESPONSE STATUS : ", runReport.status_code, "\n")
    if(runReport.status_code==200):
        print("RUN REPORT : ", json.dumps(runReport.json(), indent=5), "\n")
    else:
        print("Check server side. ERROR : ", runReport.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

RESPONSE STATUS :  200 

RUN REPORT :  {
     "uid": 3,
     "status": "FINISHED",
     "numberOfDecisions": 50,
     "numberOfRequests": 50,
     "progress": 1.0,
     "duration": 564,
     "numberOfDecisionsPerSecond": 88.65248
} 



PAUSE SIMULATION 

In [ ]:
pauseSimulationUrl = "http://simulation-server:8080/v1/simulations/{}/runs/{}/actions/pause"
simulationUid = startSimulation.json()["simulationUid"]
runUid = startSimulation.json()["uid"]

############## PAUSE SIMULATION ################
try:    
    pauseSimulation = requests.post(pauseSimulationUrl.format(simulationUid, runUid))
    print("RESPONSE STATUS : ", pauseSimulation.status_code, "\n")
    if(pauseSimulation.status_code==200):
        print("YOUR SIMULATION IS PAUSED !!! \n")
        print("RESPONSE CONTENT : ", pauseSimulation.json()["message"], "\n")
    else:
        print("Check server side. ERROR : ", pauseSimulation.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

CONTINUE TO RUN 

In [ ]:
continueSimulationUrl = "http://simulation-server:8080/v1/simulations/{}/runs/{}/actions/continue"
simulationUid = startSimulation.json()["simulationUid"]
runUid = startSimulation.json()["uid"]

############## CONTINUE SIMULATION ################
try:    
    continueSimulation = requests.post(continueSimulationUrl.format(simulationUid, runUid))
    print("RESPONSE STATUS : ", continueSimulation.status_code, "\n")
    if(continueSimulation.status_code==200):
        print("YOUR SIMULATION IS RUNNING !!! \n")
        print("RESPONSE CONTENT : ", continueSimulation.json()["message"], "\n")
    else:
        print("Check server side. ERROR : ", continueSimulation.json()["message"])
except requests.exceptions.RequestException as e:  
    raise SystemExit(e)

READ DECISION SINK

In [5]:
datasinkPath = "/Users/tiemokodembele/Documents/internShip/simulatte-public/data/ODM/loanvalidation/odm-22.0.1-miniloan-decisions-withtrace-20220721_133323-5K.json" #startSimulation.json()["dataSink"]["uri"]

outputData = []
if os.path.exists(datasinkPath):
    with open(datasinkPath) as file:
        filesize = os.path.getsize(datasinkPath)
        if filesize == 0:
            print("The file {} is empty".format(datasinkPath))
        else:
            while (line := file.readline().rstrip()):
                jsonLine = dict()
                jsonLine["request"] = json.loads(line)["request"]
                jsonLine["trace"] = json.loads(line)["trace"]
                jsonLine["response"] = json.loads(line)["response"]

                outputData.append(jsonLine)

datasink = pd.json_normalize(outputData)
#datasink = pd.json_normalize(outputData, ["taskInformation", "ruleInformation"], ["totalTasksNotExecuted", "shortname", ["tasksNotExecuted", "taskInformation"]])
datasink.head(5)

,request.loan.duration,request.loan.amount,request.loan.yearlyInterestRate,request.loan.yearlyRepayment,request.borrower.creditScore,request.borrower.name,request.borrower.yearlyIncome,request.__TraceFilter__.infoExecutionEventsAsked,request.__TraceFilter__.infoTasks,request.__TraceFilter__.infoExecutionDuration,...,trace.outputParameters.loan.messages,trace.outputParameters.loan.yearlyInterestRate,trace.outputParameters.loan.yearlyRepayment,response.loan.duration,response.loan.amount,response.loan.approved,response.loan.messages,response.loan.yearlyInterestRate,response.loan.yearlyRepayment,response.__DecisionID__
0,162,783440,0.030005,70648,736,Stephanie Roberts,113243,False,False,True,...,"[Too big Debt-To-Income ratio, debt-to-income ...",0.030005,70648,162,783440,False,"[Too big Debt-To-Income ratio, debt-to-income ...",0.030005,70648,7458b498-abc1-44ef-8138-790dd4583fdb0
1,183,1787385,0.054819,173197,527,Julie Harris,224186,False,False,True,...,"[The loan cannot exceed 1,000,000]",0.054819,173197,183,1787385,False,"[The loan cannot exceed 1,000,000]",0.054819,173197,e21d78c2-5671-4126-94cc-b161ff771cf10
2,75,1733494,0.055183,328563,460,Andre Cohen,39954,False,False,True,...,"[The loan cannot exceed 1,000,000]",0.055183,328563,75,1733494,False,"[The loan cannot exceed 1,000,000]",0.055183,328563,5280c86e-4b3c-499c-885f-ef1b3eaadb290
3,72,1350004,0.043400,255970,751,April Grimes,219998,False,False,True,...,"[The loan cannot exceed 1,000,000]",0.043400,255970,72,1350004,False,"[The loan cannot exceed 1,000,000]",0.043400,255970,a899017e-4a2a-4de7-a880-680f3c97d96d0
4,160,1210944,0.037772,115742,684,Stephen Gonzales,72470,False,False,True,...,"[The loan cannot exceed 1,000,000]",0.037772,115742,160,1210944,False,"[The loan cannot exceed 1,000,000]",0.037772,115742,1e7015c3-d8a0-48f5-8359-39a13e4d37f40


GET 
- Total of approved amounts
- Total of requested amounts 
- f(yearlyIncome)=loanAmount

In [6]:
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pgeocode

#Graph configs
approval_colors = ["#00CC96", "#FF6692"]
color_indices = [0, 1]

pie_chart_approval_loan_names = ['approved', 'rejected']
pie_chart_loan_amount_names = ['approved amount', 'rejected amount']
bidirectional_bar_chart_label_v2=['number of applications', 'sum of applications amounts']
bidirectional_bar_chart_label=['number of applications', 'sum of applications amounts']
bidirectional_bar_chart_label=['number of rejected applications', 'sum of rejected applications amounts', 'number of approved applications', 'sum of approved applications amounts']
radar_chart_labels=['number of rejected applications', 'sum of rejected applications amounts', 'number of approved applications', 'sum of approved applications amounts', 'ratio approved/requested']

Metrics

In [7]:
########### ODM #############
select_approved_decision = '`response.loan.approved` == True'
select_rejected_decision = '`response.loan.approved` == False'

get_decision_approval = 'response.loan.approved'
get_decision_amount = 'response.loan.amount'
get_decision_borrower_yearly_income = 'request.borrower.yearlyIncome'
get_decision_borrower_credit_score = "request.borrower.creditScore"
get_decision_id = 'response.__DecisionID__'
get_decision_reimbursement_rate = 'response.loan.yearlyInterestRate'
get_decision_yearly_repayment = 'response.loan.yearlyRepayment'

KPIs

In [8]:
########### KPI #############
approved_loan = datasink.query(select_approved_decision)
rejected_loan = datasink.query(select_rejected_decision)

########### Approved/Rejected PRINT #############
print("Number of approved loan request = ", approved_loan.shape[0])
print("Number of rejected loan request = ", rejected_loan.shape[0])
print("Rate of approval = ", approved_loan.shape[0]/datasink.shape[0])

print("Total approved amount = ",datasink.query(select_approved_decision)[get_decision_amount].sum())
print("Total rejected amount = ",datasink.query(select_rejected_decision)[get_decision_amount].sum())
print("Total requested amount = ",datasink[get_decision_amount].sum())

Number of approved loan request =  1087
Number of rejected loan request =  3913
Rate of approval =  0.2174
Total approved amount =  366173820
Total rejected amount =  4676578568
Total requested amount =  5042752388


DONUT CHART

In [9]:
########### settings ###########
donut_hole = 0.7

########### Approved/Rejected PIE CHART #############
trace_approval = go.Pie(labels = pie_chart_approval_loan_names,
                            values = [approved_loan.shape[0], rejected_loan.shape[0]],
                            hole = donut_hole)

########### Approved/Rejected Amount PIE CHART #############
trace_amount = go.Pie(values= [datasink.query(select_approved_decision)[get_decision_amount].sum(),
                                 datasink.query(select_rejected_decision)[get_decision_amount].sum()],
                            labels=pie_chart_loan_amount_names,
                            hole = donut_hole,
                            name="Loan amount")

fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    column_widths=[0.5, 0.5],
                    #vertical_spacing=0.001,
                    subplot_titles=("Approval", "Approved/rejected amount")
                    )

fig.add_trace(trace_approval, row=1, col=1)
fig.add_trace(trace_amount, row=1, col=2)

fig.update_traces(hoverinfo='label', textinfo='value', textfont_size=15, textposition="outside",
                  marker=dict(colors=approval_colors, line=dict(color='#FFFFFF', width=7)))
                
fig.update_annotations(yshift=20) # subplot tiltes

fig.update_layout(
    barmode='stack',
    plot_bgcolor = "white",
    #paper_bgcolor = 'lightgray',
    height=600,
    title={'text': f"Number of applications and sum of loan amounts",
          'x':0.5,
          'y':0.98,
          'xanchor': 'center',
          'font_size': 20
    },
    annotations= [
         {
            "font": {
               "size": 15
            },
            "showarrow": False,
            "text": "<b>Approval</b>",
            "x": 0.23,
            "y": 0.42
         },
         {
            "font": {
               "size": 15
            },
            "showarrow": False,
            "text": "<b>Total amount</b><br> approved/rejected",
            "x": 0.775,
            "y": 0.4
         }
      ],
      )

iplot(fig)

SCATTER CHART

In [10]:
def array_normalize(values, max_value=1):
    normalized_values = []
    for value in values:
        normalized_values.append((value*max_value)/max(values))
    return normalized_values

def set_colors(values, true_color="#00CC96", false_color="#FF6692"):
    colors = []
    for value in values:
        if(value): 
            colors.append(true_color) 
        else: 
            colors.append(false_color) 
    return colors

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                 x=datasink[get_decision_borrower_yearly_income],
                 y=datasink[get_decision_amount],
                 mode='markers',
                 marker_size=array_normalize(datasink[get_decision_borrower_credit_score], 20), 
                 marker_color=set_colors(datasink[get_decision_approval]),
                 text=datasink[get_decision_approval], 
                 hovertext=datasink[get_decision_id],
                 hovertemplate="<b>Income:</b> %{x}<br><b>Loan amount:</b> %{y}<br><b>Approval:</b> %{text}<br><b>Id:</b> %{hovertext}<extra></extra>",
                 name='V1'
))


fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        y=max(datasink.query(select_approved_decision)[get_decision_amount]/datasink.query(select_approved_decision)[get_decision_borrower_yearly_income])*np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        mode="lines",
        line=go.scatter.Line(color="#109618"),
        showlegend=False)
)
fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        y=max(datasink.query(select_approved_decision)[get_decision_amount])*np.array([1, 1]),
        mode="lines",
        line=go.scatter.Line(color="#B82E2E"),
        showlegend=False)
)


fig.update_xaxes(title_text='Yearly income')
fig.update_yaxes(title_text='Loan amount')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"Behaviours according to income and loan amounts",
          'x':0.5,
          'xanchor': 'center'
    })

iplot(fig)

INTEREST RATE

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                 x=datasink[get_decision_borrower_yearly_income]/12,
                 y=datasink[get_decision_yearly_repayment],
                 mode='markers',
                 marker_size=array_normalize(datasink[get_decision_borrower_credit_score], 20), 
                 marker_color=set_colors(datasink[get_decision_approval]),
                 text=datasink[get_decision_approval], 
                 hovertext=datasink[get_decision_id],
                 hovertemplate="<b>Income:</b> %{x}<br><b>Repayment:</b> %{y}<br><b>Approval:</b> %{text}<br><b>Id:</b> %{hovertext}<extra></extra>",
                 name='V1'
))

pente1 = max(datasink.query(select_approved_decision)[get_decision_yearly_repayment]*12/datasink.query(select_approved_decision)[get_decision_borrower_yearly_income])
fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income]/12)]),
        y=pente1*np.array([0, max(datasink[get_decision_borrower_yearly_income]/12)]),
        mode="lines",
        line=go.scatter.Line(color="#109618"),
        showlegend=False)
)
fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income]/12)]),
        y=max(datasink.query(select_approved_decision)[get_decision_yearly_repayment])*np.array([1, 1]),
        mode="lines",
        line=go.scatter.Line(color="#B82E2E"), ##B82E2E
        showlegend=False)
)

print("Rate : ",pente1)

Rate :  3.597101478816539


In [14]:
############ SCATTER CHART #############
fig = go.Figure()
max_marker_size = 30
fig.add_trace(go.Scatter(
                 x=datasink[get_decision_borrower_yearly_income],
                 y=datasink[get_decision_borrower_credit_score],
                 mode='markers',
                 marker_size=array_normalize(datasink[get_decision_amount], max_marker_size), 
                 marker_color=set_colors(datasink[get_decision_approval]),
                 text=datasink[get_decision_approval], 
                 hovertemplate="<b>Income:</b> %{x}<br><b>creditScore:</b> %{y}<br><b>Approval:</b> %{text}<extra></extra>",
                 name='V1'
))

fig.update_xaxes(title_text='Yearly income')
fig.update_yaxes(title_text='Credit score')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"creditScore and Income",
          'x':0.5,
          'xanchor': 'center'
    })

iplot(fig)

RADAR

In [16]:
fig = go.Figure()

#radar dimension normalization 
radar_max_value = 100

values = np.array([[datasink.query(select_rejected_decision).shape[0], 
                    datasink.query(select_rejected_decision)[get_decision_amount].sum(), 
                    datasink.query(select_approved_decision).shape[0], 
                    datasink.query(select_approved_decision)[get_decision_amount].sum(), 
                    datasink.query(select_approved_decision).shape[0]/(datasink.query(select_approved_decision).shape[0]+datasink.query(select_rejected_decision).shape[0])
                   ],
                   [
                    datasink.shape[0],
                    datasink[get_decision_amount].sum(),
                    datasink.shape[0],
                    datasink[get_decision_amount].sum(),
                    1
                   ]
                  ])
normed_values = (values*radar_max_value) / values.max(axis=0)

def radar_normalization(values, max_value):
    normalized_values = []
    for value in values:
        normalized_values.append((value*max_value)/max(values))
    return normalized_values

fig.add_trace(go.Scatterpolar(
      r=normed_values[0],
      theta=radar_chart_labels,
      marker_color='rgb(0,204,150)',
      fillcolor='rgba(0,204,150,0.4)',
      fill='toself',
      name='V1',
))

fig.update_layout(
  height=800,
  polar=dict(
    #bgcolor= 'rgb(239, 239, 232)',
    radialaxis=dict(
      visible=True,
      range=[0, radar_max_value],
      #linecolor='red',
      #gridcolor='red',
      #axiscolor='red',
      #tickcolor='red',
    )),
  #showlegend=False
)

fig.show()

VIOLON

In [17]:
subplot_titles = ['Approved', 'Rejected']
select_decision_approval_status = [select_approved_decision, select_rejected_decision]
get_decision_value = [get_decision_amount, get_decision_amount]
fig = go.Figure()
for idx, approval in enumerate(select_decision_approval_status):
    fig.add_trace(go.Violin(y=datasink.query(approval)[get_decision_amount],
                            name=subplot_titles[idx],
                            box_visible=True,
                            meanline_visible=True,
                            points='all'
                            ))

fig.update_yaxes(title_text='Loan amount')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"Loan amount according to approval status",
          'x':0.5,
          'xanchor': 'center'
    })

fig.show()

In [18]:
subplot_titles = ['Approved', 'Rejected']
select_decision_approval_status = [select_approved_decision, select_rejected_decision]
get_decision_value = [get_decision_amount, get_decision_amount]
fig = go.Figure()
for idx, approval in enumerate(select_decision_approval_status):
    fig.add_trace(go.Violin(y=datasink.query(approval)[get_decision_borrower_credit_score],
                            name=subplot_titles[idx],
                            box_visible=True,
                            meanline_visible=True,
                            points='all'
                            ))

fig.update_yaxes(title_text='Credit score')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"Credit score according to approval status",
          'x':0.5,
          'xanchor': 'center'
    })

fig.show()

Get __decision details__ by __DecisionID__

In [ ]:
decisionId = "45f4e697-2ad9-4884-b974-fce5b619eca30" #id can change between two run of a simulation
#decisionId = "049545f0-985e-45f2-9f72-e8c64e0cadaf0"

decision = datasink.query('`response.__DecisionID__` == @decisionId')
print("Your decision is printed in {}.json".format(decisionId))
decision.to_json("{}.json".format(decisionId))

#print(decision)
#outputData = np.array(outputData)
#print("VALUE : ", np.where(outputData["response"]["__DecisionID__"] == decisionId))
#outputData[np.where(outputData["response"]["__DecisionID__"] == decisionId)]


Your decision is printed in 45f4e697-2ad9-4884-b974-fce5b619eca30.json


TRACE OPERATION

test

In [19]:
rules_counter = dict()

f = open(f"{SIMULATTE_BASE_DIR}/data/ODM/trace-structure.json", "r")
test_data = json.loads(f.read())

executed_rules = test_data['rulesFired']['ruleInformation'] # outputData[number_of_decision]['trace']
not_executed_rules = test_data['rulesNotFired']['ruleInformation']

for executed_rule in executed_rules: 
    if executed_rule['name'] in rules_counter:
        rules_counter[executed_rule['name']] += 1
    else :
        rules_counter[executed_rule['name']] = 1

for not_executed_rule in not_executed_rules: 
    rules_counter[not_executed_rule['name']] = 0

print("RULES COUNTER : ", rules_counter)

RULES COUNTER :  {'eligibility.minimum_credit_score': 1, 'eligibility.minimum_income': 1, 'eligibility.repayment_and_score_6': 1, 'eligibility.repayment_and_score_0': 0, 'validation.maximum_amount': 0, 'eligibility.repayment_and_score_2': 0, 'eligibility.repayment_and_score_4': 0}


COUNT NUMBER OF EXECUTION OF EACH RULE

In [40]:
rules = []
rules_counter = dict()

print(len(outputData))
for number_of_decision in range(0,len(outputData)):

    if 'ruleInformation' in outputData[number_of_decision]['trace']['rulesFired']:
        executed_rules = outputData[number_of_decision]['trace']['rulesFired']['ruleInformation']
        rules.extend(executed_rules)

        for executed_rule in executed_rules: 
            if executed_rule['name'] in rules_counter:
                rules_counter[executed_rule['name']] += 1
            else :
                rules_counter[executed_rule['name']] = 1

    if 'ruleInformation' in outputData[number_of_decision]['trace']['rulesNotFired']:
        not_executed_rules = outputData[number_of_decision]['trace']['rulesNotFired']['ruleInformation']

        for not_executed_rule in not_executed_rules: 
            if not_executed_rule['name'] not in rules_counter:
                rules_counter[not_executed_rule['name']] = 0


print(rules_counter)
rules_coverage = pd.DataFrame(rules_counter.items(), columns=['Rule', 'Count'])
rules_coverage['Percentage']= (rules_coverage['Count']*100)/datasink.shape[0]

rules_coverage

5000
{'eligibility.minimum_income': 1394, 'eligibility.repayment_and_score_6': 889, 'eligibility.repayment_and_score_0': 0, 'eligibility.minimum_credit_score': 0, 'validation.maximum_amount': 2519, 'eligibility.repayment_and_score_2': 59, 'eligibility.repayment_and_score_4': 34}


,Rule,Count,Percentage
0,eligibility.minimum_income,1394,27.88
1,eligibility.repayment_and_score_6,889,17.78
2,eligibility.repayment_and_score_0,0,0.00
3,eligibility.minimum_credit_score,0,0.00
4,validation.maximum_amount,2519,50.38
5,eligibility.repayment_and_score_2,59,1.18
6,eligibility.repayment_and_score_4,34,0.68


RULES COVERAGE

In [53]:
fig = go.Figure(go.Bar(
            x=[-10 if count==0 else count for count in rules_coverage['Count']],
            y=rules_coverage["Rule"],
            text=[f'------> {item} %' if item==0.00 else f'{item} %' for item in rules_coverage['Percentage']],
            #text=['{:.2f} %'.format(item) for item in rules_coverage['Percentage']], 
            marker_color=[px.colors.qualitative.Plotly[6] if item<=0 else px.colors.qualitative.Plotly[2] for item in rules_coverage['Percentage']],
            #textposition="inside",
            hovertemplate="<b>%{y}</b><br><b>Count:</b> %{x}<br><b>Percentage:</b> %{text}<extra></extra>",
            orientation='h'))

fig.update_xaxes(title_text='Number of decisions')
fig.update_yaxes(title_text='Rules')

fig.update_layout(
    title='RULES COVERAGE',
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=50, t=70, b=70),
    height=700,
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)
fig.show()